# Projeto 1 - Classificador Naive-Bayes com Tweets


## Objetivo

* Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa selecionealgumas mensagens disponíveis no Twitter que mencionam esse produto e classifique-as como"relevante" ou "irrelevante".

* Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.

* Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamentedo Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails, porexemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.

* Para realizar o MVP (minimum viable product) do projeto, você precisa implementar umaversão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.

* Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Integrantes do Projeto

**Nome:** Wilgner Lopes Araujo

**Nome:** Fabricio Neri Lima

## Informações relevantes

* Esse arquivo representa a ultima etapa do projeto onde haverá todas as etapas de cleaning da base de dados e implantação do algoritmo de classificação

* O arquivo que pesquisa os tweets esta na pasta Arquivo Jupyter/Projeto1_Obtenção_dos_tweets nessa mesma raiz do repositório

* A base de dados se encontra na pasta dados/Valorant.xlsx

* A conta utiliza no twitter para obtenção de dados é @AnalyticsInsper

* O produto escolhido para o projeto é : **Valorant**

* Os criterios para a classificação dos tweets na primeira etapa do projeto foram: Os tweets que falavam de alguma forma positiva ou negativa sobre o jogo ou até mesmo que relacionavam o jogo a alguma emoção, foram classificados como relevantes. Os demais foram classificados como Irrelevantes.


# Começando

## Declarando as bibliotecas

In [1]:
import math
import numpy as np
import pandas as pd
import re

## Abrindo a base de dados e visualizando os primeiros 5 termos

In [2]:
dados = pd.read_excel('../dados/Valorant.xlsx')
dados.head()

,Treinamento,Etiquetas,Classificado
0,@caos__0 eu no valorant :v\nfon,2,Irrelevante
1,o cara q pra mim é o melhor player de valorant...,2,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante
3,"deletei o valorant, agora meu pc volta a ligar...",1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu...,1,Relevante


## Informações do DataFrame

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Treinamento   500 non-null    object
 1   Etiquetas     500 non-null    int64 
 2   Classificado  500 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


## Classificando 

* A coluna Treinamento e Classificado são categoricas, pois ambas são variaveis qualitativas nominais

In [4]:
dados['Treinamento'] = dados['Treinamento'].astype('category')
dados['Classificado'] = dados['Classificado'].astype('category')

## Iniciando as coletas de probabilidades

### Analisando frequencia de Relevante e Irrelevante na base de dados

In [5]:
Probabilidade_total = dados.Classificado.value_counts(True)
Probabilidade_total

Irrelevante    0.542
Relevante      0.458
Name: Classificado, dtype: float64

## Declarando as variaveis de probabilidade

* Considere P(R) para probabilidade de ser Relevante
* Considere P(R_c) para probabilidade de ser Irrelevante

In [6]:
P_R_c = Probabilidade_total[0]
P_R = Probabilidade_total[1]

## Separando a base de dados em Relevantes e Irrelevantes

In [7]:
filtro_relevante = dados.Classificado == 'Relevante'
filtro_irrelevante = dados.Classificado == 'Irrelevante'

dados_relevantes = dados.loc[filtro_relevante,:]
dados_irrelevantes = dados.loc[filtro_irrelevante,:]

In [8]:
dados_relevantes.head()

,Treinamento,Etiquetas,Classificado
3,"deletei o valorant, agora meu pc volta a ligar...",1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu...,1,Relevante
5,voltei a jogar valorant so por causa do mata mata,1,Relevante
6,viciei em valorant esse é o tweet,1,Relevante
7,@zncabral_ free fire fez mais sucesso que o va...,1,Relevante


## Antes de prosseguir vamos limpar o codigo

* Será removido os seguintes simbolos: \n, :, ", ', (,), ., ;, :, ?, !, <, >, 

In [ ]:
def limpa_texto(text):
    
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed